## Loading Packages

In [ ]:
import os
import sys
import h5py
import glob
import json
import math
import time
import timeit
import hdbscan
import itertools
import importlib
import subprocess
import numpy as np
import pandas as pd
import seaborn as sns
import xgboost as xgb
import tensorflow as tf
from scipy import stats
import keras.backend as K
import matplotlib.pyplot as plt
from collections import defaultdict
from sklearn.cluster import DBSCAN, KMeans
from scipy.stats import shapiro, mannwhitneyu, ttest_ind, spearmanr
from sklearn.preprocessing import normalize, scale, MinMaxScaler, StandardScaler
from scipy.cluster.hierarchy import single, complete, average, ward, dendrogram, linkage, fcluster

from tcn import TCN
from pandas import read_csv
from sklearn.svm import SVC
from sklearn import svm, datasets
from IPython.display import Image
from sklearn.tree import plot_tree
from sklearn.decomposition import PCA
from tensorflow.keras.optimizers import Adam
from sklearn.neighbors import NearestNeighbors
from tensorflow.keras.utils import to_categorical
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from tensorflow.keras.metrics import Precision, Recall
from sktime.transformations.panel.rocket import Rocket
from sklearn.metrics.cluster import normalized_mutual_info_score
from tensorflow.keras.models import Model, load_model, Sequential
from sklearn.linear_model import LogisticRegression, RidgeClassifierCV
from sklearn.model_selection import train_test_split, TimeSeriesSplit, KFold
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, BatchNormalization, Activation, GlobalAveragePooling1D, Dense, add, Dropout, concatenate, LSTM
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score, roc_auc_score, roc_curve, auc, log_loss, classification_report, silhouette_score

## Prelims

In [ ]:
df = pd.read_csv('')
df = df.drop(df.columns[0], axis=1)
data = df.iloc[:, :-2]
data = data.iloc[:-1, :]
data.tail(20)

In [ ]:
# full dataset

df_all = df.iloc[:-1, :]  
predictor_columns = df_all.iloc[:, :-2]
X_full = predictor_columns.values   # Features
y_full = df['Fault_Status'].values  # Labels

# Normalize features
scaler = MinMaxScaler(feature_range=(0, 1))
X_full = scaler.fit_transform(X_full)

## TSCV Benchmarking for Fault Detection

In [ ]:
# plotting parameters
SMALL_SIZE = 8
MEDIUM_SIZE = 15
BIGGER_SIZE = 25

plt.rc('font', size=BIGGER_SIZE)          
plt.rc('axes', titlesize=MEDIUM_SIZE)     
plt.rc('axes', labelsize=BIGGER_SIZE)      
plt.rc('xtick', labelsize=MEDIUM_SIZE)    
plt.rc('ytick', labelsize=MEDIUM_SIZE)    
plt.rc('legend', fontsize=SMALL_SIZE)             
plt.rc('figure', titlesize=MEDIUM_SIZE)   

In [ ]:
def build_xgb_model():
    return xgb.XGBClassifier(objective='binary:logistic', eval_metric='logloss', use_label_encoder=False)

def build_rf_model():
    return RandomForestClassifier(n_estimators=100, random_state=42)

def build_svm_model():
    return SVC(kernel='rbf', C=1.0, gamma='auto')

def build_lstm_fcn_model(input_shape):
    input_layer = Input(shape=input_shape)

    # LSTM part
    lstm_out = LSTM(64, return_sequences=True)(input_layer)
    lstm_out = BatchNormalization()(lstm_out)

    # FCN part
    conv1 = Conv1D(filters=64, kernel_size=8, padding='same')(input_layer)
    conv1 = BatchNormalization()(conv1)
    conv1 = Activation('relu')(conv1)
    
    conv2 = Conv1D(filters=64, kernel_size=5, padding='same')(conv1)
    conv2 = BatchNormalization()(conv2)
    conv2 = Activation('relu')(conv2)
    
    conv3 = Conv1D(filters=64, kernel_size=3, padding='same')(conv2)
    conv3 = BatchNormalization()(conv3)
    conv3 = Activation('relu')(conv3)
    
    # Combining LSTM and FCN parts
    x = concatenate([lstm_out, conv3])
    x = GlobalAveragePooling1D()(x)
    
    output_layer = Dense(1, activation='sigmoid')(x)
    
    model = Model(inputs=input_layer, outputs=output_layer)
    model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

def inception_block(input_tensor, filters):
    conv1 = Conv1D(filters=filters[0], kernel_size=1, activation='relu', padding='same')(input_tensor)
    conv3 = Conv1D(filters=filters[1], kernel_size=3, activation='relu', padding='same')(input_tensor)
    conv5 = Conv1D(filters=filters[2], kernel_size=5, activation='relu', padding='same')(input_tensor)
    pool = MaxPooling1D(pool_size=3, strides=1, padding='same')(input_tensor)
    pool_proj = Conv1D(filters=filters[3], kernel_size=1, activation='relu', padding='same')(pool)
    return concatenate([conv1, conv3, conv5, pool_proj], axis=-1)

def build_inception_model(input_shape):
    input_layer = Input(shape=input_shape)
    x = Conv1D(64, 7, strides=2, padding='same', activation='relu')(input_layer)
    x = MaxPooling1D(3, strides=2, padding='same')(x)
    x = inception_block(x, [64, 128, 32, 32])
    x = inception_block(x, [128, 192, 96, 64])
    x = GlobalAveragePooling1D()(x)
    x = Dropout(0.4)(x)
    output_layer = Dense(1, activation='sigmoid')(x)  
    model = Model(inputs=input_layer, outputs=output_layer)
    model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])
    return model

def build_resnet(input_shape):
    n_feature_maps = 64

    input_layer = Input(shape=input_shape)

    # BLOCK 1
    conv_x = Conv1D(filters=n_feature_maps, kernel_size=8, padding='same')(input_layer)
    conv_x = BatchNormalization()(conv_x)
    conv_x = Activation('relu')(conv_x)

    conv_y = Conv1D(filters=n_feature_maps, kernel_size=5, padding='same')(conv_x)
    conv_y = BatchNormalization()(conv_y)
    conv_y = Activation('relu')(conv_y)

    conv_z = Conv1D(filters=n_feature_maps, kernel_size=3, padding='same')(conv_y)
    conv_z = BatchNormalization()(conv_z)

    shortcut_y = Conv1D(filters=n_feature_maps, kernel_size=1, padding='same')(input_layer)
    shortcut_y = BatchNormalization()(shortcut_y)

    output_block_1 = add([shortcut_y, conv_z])
    output_block_1 = Activation('relu')(output_block_1)

    # BLOCK 2
    conv_x = Conv1D(filters=n_feature_maps * 2, kernel_size=8, padding='same')(output_block_1)
    conv_x = BatchNormalization()(conv_x)
    conv_x = Activation('relu')(conv_x)

    conv_y = Conv1D(filters=n_feature_maps * 2, kernel_size=5, padding='same')(conv_x)
    conv_y = BatchNormalization()(conv_y)
    conv_y = Activation('relu')(conv_y)

    conv_z = Conv1D(filters=n_feature_maps * 2, kernel_size=3, padding='same')(conv_y)
    conv_z = BatchNormalization()(conv_z)

    shortcut_y = Conv1D(filters=n_feature_maps * 2, kernel_size=1, padding='same')(output_block_1)
    shortcut_y = BatchNormalization()(shortcut_y)

    output_block_2 = add([shortcut_y, conv_z])
    output_block_2 = Activation('relu')(output_block_2)

    # BLOCK 3
    conv_x = Conv1D(filters=n_feature_maps * 2, kernel_size=8, padding='same')(output_block_2)
    conv_x = BatchNormalization()(conv_x)
    conv_x = Activation('relu')(conv_x)

    conv_y = Conv1D(filters=n_feature_maps * 2, kernel_size=5, padding='same')(conv_x)
    conv_y = BatchNormalization()(conv_y)
    conv_y = Activation('relu')(conv_y)

    conv_z = Conv1D(filters=n_feature_maps * 2, kernel_size=3, padding='same')(conv_y)
    conv_z = BatchNormalization()(conv_z)

    shortcut_y = BatchNormalization()(output_block_2)

    output_block_3 = add([shortcut_y, conv_z])
    output_block_3 = Activation('relu')(output_block_3)

    gap_layer = GlobalAveragePooling1D()(output_block_3)

    output_layer = Dense(1, activation='sigmoid')(gap_layer)

    model = Model(inputs=input_layer, outputs=output_layer)
    
    model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])

    return model

def build_tcn_model(input_shape):
    
    input_layer = Input(shape=input_shape)
    tcn_layer = TCN(nb_filters=64, kernel_size=3, dilations=[1, 2, 4, 8], padding='causal', return_sequences=True)(input_layer)

    pooling_layer = GlobalAveragePooling1D()(tcn_layer)
    output_layer = Dense(1, activation='sigmoid')(pooling_layer)
    model = Model(inputs=input_layer, outputs=output_layer)
    
    model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

    return model

In [ ]:
# skipping mechanism
if len(np.unique(y_train)) < 2 or len(np.unique(y_test)) < 2:  
            print(f"Skipping split {i+1} as it does not contain both classes.")
            continue

In [ ]:
# wf with skipping mechanisim

def run_expanding_window_tscv(model_build_fn, model_name, X, y, n_splits, save_path, epochs=100):
    tscv = TimeSeriesSplit(n_splits=n_splits)
    results = []
    splits = []
    for i, (train_index, test_index) in enumerate(tscv.split(X)):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        unique, counts_train = np.unique(y_train, return_counts=True)
        train_distribution = dict(zip(unique, counts_train))
        
        unique, counts_test = np.unique(y_test, return_counts=True)
        test_distribution = dict(zip(unique, counts_test))
        
        splits.append((train_index, test_index))
        
        print(f"Split {i+1}:")
        print(f"  Training set class distribution: {train_distribution}")
        print(f"  Testing set class distribution: {test_distribution}")
        print("-" * 50)
        
        if len(np.unique(y_train)) < 2 or len(np.unique(y_test)) < 2:  
            print(f"Skipping split {i+1} as it does not contain both classes.")
            continue

        if model_name in ["LSTM_FCN", "InceptionTime", "ResNet", "TCN"]:
            X_train, X_test = np.expand_dims(X_train, axis=-1), np.expand_dims(X_test, axis=-1)
            model = model_build_fn(input_shape=(X_train.shape[1], X_train.shape[2]))
        else:
            model = model_build_fn()
        
        start_time = timeit.default_timer()
        if model_name in ["LSTM_FCN", "InceptionTime", "ResNet", "TCN"]:
            model.fit(X_train, y_train, epochs=epochs, verbose=0)  
        else:
            model.fit(X_train, y_train)
        train_time = timeit.default_timer() - start_time
        
        start_time = timeit.default_timer()
        if hasattr(model, 'predict_proba'):
            y_probs = model.predict_proba(X_test)[:, 1]
        else:
            y_probs = model.predict(X_test).squeeze()  # For models that output probabilities directly
        elapsed = timeit.default_timer() - start_time

        roc_auc = roc_auc_score(y_test, y_probs)
        
        results.append({
            'split': i+1,
            'roc_auc_score': roc_auc
        })
        K.clear_session()  

    results_df = pd.DataFrame(results)
    results_df.to_csv(save_path, index=False)
    print(f"Results saved to {save_path}")

models = {
    "XGBoost": build_xgb_model,
    "RandomForest": build_rf_model,
    "SupportVectorMachine": build_svm_model,
    "LSTM_FCN": build_lstm_fcn_model,
    "InceptionTime": build_inception_model,
    "ResNet": build_resnet,
    "TCN": build_tcn_model
}

X = X_full
y = y_full

save_path = ''
for n_splits in range(3, 10):  # From 3 to 9 splits
    for model_name, model_fn in models.items():
        model_save_path = f"{save_path}/{model_name}_splits_{n_splits}.csv"
        print(f"Running {model_name} with {n_splits} splits...")
        if model_name in ["LSTM_FCN", "InceptionTime", "ResNet", "TCN"]:
            run_expanding_window_tscv(model_fn, model_name, X, y, n_splits, model_save_path, epochs=100)
        else:
            run_expanding_window_tscv(model_fn, model_name, X, y, n_splits, model_save_path)

In [ ]:
def run_expanding_window_tscv(model_build_fn, model_name, X, y, n_splits, save_path, epochs=100):
    tscv = TimeSeriesSplit(n_splits=n_splits)
    results = []
    splits = []
    for i, (train_index, test_index) in enumerate(tscv.split(X)):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        unique, counts_train = np.unique(y_train, return_counts=True)
        train_distribution = dict(zip(unique, counts_train))
        
        unique, counts_test = np.unique(y_test, return_counts=True)
        test_distribution = dict(zip(unique, counts_test))
        
        splits.append((train_index, test_index))
        
        print(f"Split {i+1}:")
        print(f"  Training set class distribution: {train_distribution}")
        print(f"  Testing set class distribution: {test_distribution}")
        print("-" * 50)
        
        if model_name in ["LSTM_FCN", "InceptionTime", "ResNet", "TCN"]:
            X_train, X_test = np.expand_dims(X_train, axis=-1), np.expand_dims(X_test, axis=-1)
            model = model_build_fn(input_shape=(X_train.shape[1], X_train.shape[2]))
        else:
            model = model_build_fn()
        
        start_time = timeit.default_timer()
        if model_name in ["LSTM_FCN", "InceptionTime", "ResNet", "TCN"]:
            model.fit(X_train, y_train, epochs=epochs, verbose=0)  
        else:
            model.fit(X_train, y_train)
        elapsed = timeit.default_timer() - start_time
    
        if hasattr(model, 'predict_proba'):
            y_probs = model.predict_proba(X_test)[:, 1]
        else:
            y_probs = model.predict(X_test).squeeze()  # For models that output probabilities directly

        roc_auc = roc_auc_score(y_test, y_probs)
        
        results.append({
            'split': i+1,
            'roc_auc_score': roc_auc,
            'time_to_prediction': elapsed
        })
        K.clear_session()  

    results_df = pd.DataFrame(results)
    results_df.to_csv(save_path, index=False)
    print(f"Results saved to {save_path}")

models = {
    "XGBoost": build_xgb_model,
    "RandomForest": build_rf_model,
    "SupportVectorMachine": build_svm_model,
    "LSTM_FCN": build_lstm_fcn_model,
    "InceptionTime": build_inception_model,
    "ResNet": build_resnet,
    "TCN": build_tcn_model
}

X = X_full
y = y_full

save_path = ''
for n_splits in range(3, 10):  # From 3 to 9 splits
    for model_name, model_fn in models.items():
        model_save_path = f"{save_path}/{model_name}_splits_{n_splits}.csv"
        print(f"Running {model_name} with {n_splits} splits...")
        if model_name in ["LSTM_FCN", "InceptionTime", "ResNet", "TCN"]:
            run_expanding_window_tscv(model_fn, model_name, X, y, n_splits, model_save_path, epochs=100)
        else:
            run_expanding_window_tscv(model_fn, model_name, X, y, n_splits, model_save_path)

In [ ]:
def run_sliding_window_tscv(model_build_fn, model_name, X, y, n_splits, test_size, save_path, epochs=100):
    total_size = len(X)
    train_size = total_size - (n_splits * test_size)  

    results = []
    splits = []
    for i in range(n_splits):
        start_train = i * test_size
        end_train = start_train + train_size
        start_test = end_train
        end_test = start_test + test_size

        # Ensure we do not go out of bounds
        if end_test > total_size:
            break

        train_index = np.arange(start_train, end_train)
        test_index = np.arange(start_test, end_test)

        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        unique, counts_train = np.unique(y_train, return_counts=True)
        train_distribution = dict(zip(unique, counts_train))
        
        unique, counts_test = np.unique(y_test, return_counts=True)
        test_distribution = dict(zip(unique, counts_test))
        
        splits.append((train_index, test_index))
        
        print(f"Split {i+1}:")
        print(f"  Training set class distribution: {train_distribution}")
        print(f"  Testing set class distribution: {test_distribution}")
        print("-" * 50)

        if len(np.unique(y_train)) < 2 or len(np.unique(y_test)) < 2:  
            print(f"Warning: Only one class present in y_test for Split {i+1}, skipping ROC AUC calculation.")
            roc_auc = 0  # Assign NaN or a specific value if you prefer
            continue
        else:
            if model_name in ["LSTM_FCN", "InceptionTime", "ResNet", "TCN"]:
                X_train, X_test = np.expand_dims(X_train, axis=-1), np.expand_dims(X_test, axis=-1)
                model = model_build_fn(input_shape=(X_train.shape[1], X_train.shape[2]))
            else:
                model = model_build_fn()
            
            #start_time = timeit.default_timer()
            if model_name in ["LSTM_FCN", "InceptionTime", "ResNet", "TCN"]:
                model.fit(X_train, y_train, epochs=epochs, verbose=0)  
            else:
                model.fit(X_train, y_train)
            #elapsed = timeit.default_timer() - start_time
            
            y_pred = (model.predict(X_test) > 0.5).astype(int)
            #y_pred = (model.predict(X_test))

            if hasattr(model, 'predict_proba'):
                y_probs = model.predict_proba(X_test)[:, 1]
            else:
                y_probs = model.predict(X_test).squeeze()  # For models that output probabilities directly

        roc_auc = roc_auc_score(y_test, y_probs)
        
        results.append({
            'split': i+1,
            'roc_auc_score': roc_auc
        })
        K.clear_session()  

    results_df = pd.DataFrame(results)
    results_df.to_csv(save_path, index=False)
    print(f"Results saved to {save_path}")

X = X_full
y = y_full

save_path = ''
models = {
    "XGBoost": build_xgb_model,
    "RandomForest": build_rf_model,
    "SupportVectorMachine": build_svm_model,
    "LSTM_FCN": build_lstm_fcn_model,
    "InceptionTime": build_inception_model,
    "ResNet": build_resnet,
    "TCN": build_tcn_model
}

for n_splits in range(3,10):  # From 3 to 9 splits
    test_size = 150  
    for model_name, model_fn in models.items():
        model_save_path = f"{save_path}/{model_name}_splits_{n_splits}.csv"
        print(f"Running {model_name} with {n_splits} splits...")
        if model_name in ["LSTM_FCN", "InceptionTime", "ResNet", "TCN"]:
            run_sliding_window_tscv(model_fn, model_name, X, y, n_splits, test_size, model_save_path, epochs=100)
        else:
            run_sliding_window_tscv(model_fn, model_name, X, y, n_splits, test_size,model_save_path)

### ROCKET

In [ ]:
# wf
def convert_to_nested_df(X):
    scaler = MinMaxScaler(feature_range=(0, 1))
    X_scaled = scaler.fit_transform(X)
    X_scaled = pd.DataFrame(X_scaled)
    nested_df = pd.DataFrame({'series': [pd.Series(X_scaled.iloc[i, :]) for i in range(X_scaled.shape[0])]})
    return nested_df

def walk_forward_tscv(X, y, n_splits=7):
    tscv = TimeSeriesSplit(n_splits=n_splits)
    results = []

    # Assuming X is already a DataFrame at this point; if not, it should be converted before this function
    X_nested = convert_to_nested_df(X)
    
    rocket = Rocket(random_state=42)
    classifier = LogisticRegression()

    for i, (train_index, test_index) in enumerate(tscv.split(X_nested)):
        X_train, X_test = X_nested.iloc[train_index], X_nested.iloc[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        if len(np.unique(y_train)) < 2 or len(np.unique(y_test)) < 2:  # 
            print(f"Skipping split {i+1} as training/testing does not contain both classes.")
            continue

        rocket.fit(X_train)
        X_train_transform = rocket.transform(X_train)
        X_test_transform = rocket.transform(X_test)

        classifier.fit(X_train_transform, y_train)
        y_pred = classifier.predict(X_test_transform)
        #y_pred_proba = classifier.predict_proba(X_test_transform)[:, 1]

        roc_auc = roc_auc_score(y_test, y_pred)
        #results.append(roc_auc)
        results.append({'split': i + 1, 'roc_auc_score': roc_auc})
        print(f"Split ROC-AUC: {roc_auc}")
    
    return results

def save_results(results, file_path):
    results_df = pd.DataFrame(results)
    results_df.to_csv(file_path, index=False)
    print(f"Results saved to {file_path}")

X = X_full
y = y_full

save_path_base = ''
model_name = "ROCKET"

# Running the TSCV for the ROCKET model
for n_splits in range(3, 10):  # From 3 to 6 splits
    model_save_path = f"{save_path_base}/{model_name}_splits_{n_splits}.csv"
    print(f"Running {model_name} with {n_splits} splits...")
    results = walk_forward_tscv(X, y, n_splits)
    save_results(results, model_save_path)

In [ ]:
# sw 

def save_results(results, file_path):
    results_df = pd.DataFrame(results)
    results_df.to_csv(file_path, index=False)
    print(f"Results saved to {file_path}")

def convert_to_nested_df(X):
    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X)
    X_scaled = pd.DataFrame(X_scaled)
    nested_df = pd.DataFrame({'series': [pd.Series(X_scaled.iloc[i, :]) for i in range(X_scaled.shape[0])]})
    return nested_df

def sliding_window_tscv(X, y, n_splits=7, test_size=150):
    total_size = len(X)
    train_size = total_size - (n_splits * test_size)
    splits = []
    
    for i in range(n_splits):
        start_train = i * test_size
        end_train = start_train + train_size
        start_test = end_train
        end_test = start_test + test_size
        
        if end_test > total_size:
            break
        
        train_index = np.arange(start_train, end_train)
        test_index = np.arange(start_test, end_test)
        splits.append((train_index, test_index))
        
    return splits

def run_rocket_with_tscv(X, y, splits, save_path):
    X_scaled = convert_to_nested_df(X)
    results = []

    for i, (train_index, test_index) in enumerate(splits):
        print(f"Processing split {i+1}/{len(splits)}")
        
        X_train, X_test = X_scaled.iloc[train_index], X_scaled.iloc[test_index]
        y_train, y_test = y[train_index], y[test_index]

        if len(np.unique(y_train)) < 2 or len(np.unique(y_test)) < 2:  
            print(f"Skipping split {i+1} as it does not contain both classes.")
            continue  

        rocket = Rocket(random_state=42)
        rocket.fit(X_train)
        X_train_transform = rocket.transform(X_train)
        X_test_transform = rocket.transform(X_test)

        classifier = LogisticRegression()
        classifier.fit(X_train_transform, y_train)
        y_pred_proba = classifier.predict_proba(X_test_transform)[:, 1]

        roc_auc = roc_auc_score(y_test, y_pred_proba)
        results.append({'split': i + 1, 'roc_auc_score': roc_auc})

    results_df = pd.DataFrame(results)
    results_df.to_csv(save_path, index=False)
    print(f"Results saved to {save_path}")

X = X_full
y = y_full

save_path_base = ''
model_name = "ROCKET"

for n_splits in range(3, 10):  # From 3 to 9 splits
    model_save_path = f"{save_path_base}/{model_name}_splits_{n_splits}.csv"
    print(f"Running {model_name} with {n_splits} splits...")
    splits = sliding_window_tscv(X, y, n_splits, test_size=150)
    results = run_rocket_with_tscv(X, y, splits, model_save_path)
    save_results(results, model_save_path)
    print(f"Results for {n_splits} splits saved to {model_save_path}")